In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import GridSearchCV

In [2]:
# Code written by Joseph Nelson.

def interpret_dftest(dftest):
    dfoutput = pd.Series(dftest[0:2], index=['Test Statistic','p-value'])
    return dfoutput

In [8]:
df3 = pd.read_csv('data/sams_clean_data.csv')
df3.set_index('yyyymmdd', inplace = True)


In [9]:
df3.head()

,lat,long,station_id,acton_id,station_name,elevation,wteq_amt,wteq_med,wteq_amt_pct_med,prec_wytd_amt,...,"LOWER COLUMBIA, HOOD RIVER",OWYHEE MALHEUR,"RAFT, GOOSE, SALMON FALLS, BRUNEAU",SNAKE ABOVE PALISADES,"UMATILLA, WALLA WALLA, WILLOW",UPPER CLARK FORK RIVER BASIN,"WEISER, PAYETTE, BOISE","WHITE, GREEN, CEDAR, SKYKOMISH, SNOQUALMI, BAKER, SKAGIT",WILLAMETTE,"YAKIMA, AHTANUM"
yyyymmdd,,,,,,,,,,,,,,,,,,,,,
2011-02-10,48.566667,-115.450000,311,15A08S,Banfield Mountain,5600,12.1,12.2,99.0,17.5,...,0,0,0,0,0,0,0,0,0,0
2011-02-10,48.300000,-116.066667,323,16A08S,Bear Mountain,5400,37.4,38.6,97.0,54.2,...,0,0,0,0,0,0,0,0,0,0
2011-02-10,48.983333,-115.816667,918,15A05S,Garver Creek,4250,7.7,6.9,112.0,12.2,...,0,0,0,0,0,0,0,0,0,0
2011-02-10,48.916667,-114.766667,500,14A11S,Grave Creek,4300,13.1,11.4,115.0,18.6,...,0,0,0,0,0,0,0,0,0,0
2011-02-10,48.300000,-114.833333,510,14A14S,Hand Creek,5035,9.6,7.8,123.0,14.7,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df2 = pd.read_csv('basin_2000_2010.csv')
df2.set_index('yyyymmdd', inplace = True)

In [14]:
df2.head()

,lat,long,station_id,acton_id,station_name,elevation,wteq_amt,wteq_med,wteq_amt_pct_med,prec_wytd_amt,...,"LOWER COLUMBIA, HOOD RIVER",OWYHEE MALHEUR,"RAFT, GOOSE, SALMON FALLS, BRUNEAU",SNAKE ABOVE PALISADES,"UMATILLA, WALLA WALLA, WILLOW",UPPER CLARK FORK RIVER BASIN,"WEISER, PAYETTE, BOISE","WHITE, GREEN, CEDAR, SKYKOMISH, SNOQUALMI, BAKER, SKAGIT",WILLAMETTE,"YAKIMA, AHTANUM"
yyyymmdd,,,,,,,,,,,,,,,,,,,,,
2009-02-10,48.566667,-115.450000,311,15A08S,Banfield Mountain,5600,11.4,12.2,93.0,13.6,...,0,0,0,0,0,0,0,0,0,0
2009-02-10,48.300000,-116.066667,323,16A08S,Bear Mountain,5400,27.6,38.6,72.0,34.5,...,0,0,0,0,0,0,0,0,0,0
2009-02-10,48.983333,-115.816667,918,15A05S,Garver Creek,4250,6.3,6.9,91.0,9.6,...,0,0,0,0,0,0,0,0,0,0
2009-02-10,48.916667,-114.766667,500,14A11S,Grave Creek,4300,9.3,11.4,82.0,18.8,...,0,0,0,0,0,0,0,0,0,0
2009-02-10,48.300000,-114.833333,510,14A14S,Hand Creek,5035,10.7,7.8,137.0,13.6,...,0,0,0,0,0,0,0,0,0,0
